In [ ]:
%pip install transformers
%pip install tqdm

In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model_hf = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")
sd_hf = model_hf.state_dict()
for k, v in sd_hf.items():
    print(k, v.shape)

# 50257 x 768 (lookup table for tokens)
# 50257 = # of tokens
# 768 = dimension of each token

#1024 x 768 (lookup table for position)
# 1024 = max sequence length, used for where the word is placed
# 768 = dimension of each token



In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
# Sampling
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='openai-community/gpt2')
set_seed(42)

generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

In [ ]:
# how to train the model -> train it off tiny shakespeare dataset 
with open('../neurals/dataset/tinyShakespeare.txt', 'r') as f:
    text = f.read()

data = text[:1000]
print(data[:100])


In [ ]:
import tiktoken 
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
tokens[:10]

In [ ]:
import torch 
buf = torch.tensor(tokens[:24 + 1])

# batch our tokens 
x = buf[:-1].view(4,6)
y = buf[1:].view(4,6)

x,y

In [ ]:
from datasets import load_dataset
import numpy as np
import multiprocessing as mp
import os
import tiktoken

In [ ]:
db = load_dataset("databricks/databricks-dolly-15k")

In [ ]:
db.column_names

In [ ]:
enc = tiktoken.get_encoding('gpt2')
eot = enc._special_tokens['<|endoftext|>'] # end of text token

def tokenize(doc):
    txt = doc["context"] +  doc["instruction"] + doc["response"]
    tokens = [eot]
    tokens.extend(enc.encode_ordinary(txt))
    tokens_np = np.array(tokens)
    tokens_np_uint16 = tokens_np.astype(np.uint16)
    return tokens_np_uint16

nprocs = max(1, os.cpu_count()//2)
with mp.Pool(nprocs) as pool:
    for tokens in pool.imap(tokenize, db['train'], chunksize=15):
        print(tokens)

In [ ]:
import numpy as np
# breaking at shard 3


z = np.load('dolly_dataset/dolly_train_000001.npy')
z.shape

100000/(16*1024)